Problem 1

In [1]:
import pandas as pd

#a
auto = pd.read_csv("Auto.csv", na_values = '?')
# check for some item
auto.iloc[32]

mpg                     25
cylinders                4
displacement            98
horsepower             NaN
weight                2046
acceleration            19
year                    71
origin                   1
name            ford pinto
Name: 32, dtype: object

In [2]:
#b
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
df_quant = auto[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']]
scatter_matrix(df_quant, alpha = 0.3, figsize = (6,6), diagonal = 'kde')
plt.show()

<Figure size 600x600 with 64 Axes>

In [3]:
#c
df_quant.corr()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
mpg,1.000000,-0.776260,-0.804443,-0.778427,-0.831739,0.422297,0.581469,0.563698
cylinders,-0.776260,1.000000,0.950920,0.842983,0.897017,-0.504061,-0.346717,-0.564972
displacement,-0.804443,0.950920,1.000000,0.897257,0.933104,-0.544162,-0.369804,-0.610664
horsepower,-0.778427,0.842983,0.897257,1.000000,0.864538,-0.689196,-0.416361,-0.455171
weight,-0.831739,0.897017,0.933104,0.864538,1.000000,-0.419502,-0.307900,-0.581265
acceleration,0.422297,-0.504061,-0.544162,-0.689196,-0.419502,1.000000,0.282901,0.210084
year,0.581469,-0.346717,-0.369804,-0.416361,-0.307900,0.282901,1.000000,0.184314
origin,0.563698,-0.564972,-0.610664,-0.455171,-0.581265,0.210084,0.184314,1.000000


In [4]:
#d
import statsmodels.api as sm

X = df_quant.drop(columns = ['mpg'])
X1 = sm.add_constant(X)
X1.head(10)

/Users/zhouanqi/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,const,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,1.0,8,307.0,130.0,3504,12.0,70,1
1,1.0,8,350.0,165.0,3693,11.5,70,1
2,1.0,8,318.0,150.0,3436,11.0,70,1
3,1.0,8,304.0,150.0,3433,12.0,70,1
4,1.0,8,302.0,140.0,3449,10.5,70,1
5,1.0,8,429.0,198.0,4341,10.0,70,1
6,1.0,8,454.0,220.0,4354,9.0,70,1
7,1.0,8,440.0,215.0,4312,8.5,70,1
8,1.0,8,455.0,225.0,4425,10.0,70,1
9,1.0,8,390.0,190.0,3850,8.5,70,1


In [5]:
regression = sm.OLS(auto['mpg'], X1, missing = 'drop')
result = regression.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     252.4
Date:                Tue, 18 Feb 2020   Prob (F-statistic):          2.04e-139
Time:                        09:25:54   Log-Likelihood:                -1023.5
No. Observations:                 392   AIC:                             2063.
Df Residuals:                     384   BIC:                             2095.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -17.2184      4.644     -3.707   

As we can see from the above results, the coefficients of displacement, weight, year, and origin variables are all statistically significant at a 1% level. And the coefficients of cylinders, horsepower and acceleration variables are even not statistically at the 10% level.

Interpretation: The number of miles a vehicle can run with per gallon usage will increase 0.7508 on average for one unit increase in vehicle's year of model, assuming all other variables are fixed.  

In [6]:
#e

I think three variables that look most likely to have a nonlinear relationship with mpg are displacement, horsepower, and weight.

In [7]:
X['displacement^2'] = X['displacement']**2
X['horsepower^2'] = X['horsepower']**2
X['weight^2'] = X['weight']**2
X['acceleration^2'] = X['acceleration']**2

X2 = sm.add_constant(X)
regression2 = sm.OLS(auto['mpg'], X2, missing = 'drop')
result2 = regression2.fit()

print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                     230.2
Date:                Tue, 18 Feb 2020   Prob (F-statistic):          1.75e-160
Time:                        09:25:56   Log-Likelihood:                -962.02
No. Observations:                 392   AIC:                             1948.
Df Residuals:                     380   BIC:                             1996.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             20.1084      6.696      3.

The adjusted R square now is 0.866, which is better than 0.818 in (d).

The statistical significance of displacement coefficient is not significant at 1% level now, and it's even not significant at 10% level, as well as the coefficient on its squared term.

The p-value of cylinder coefficient is even larger, increasing from 0.128 to 0.440, so it become more statistically insignificant.

In [8]:
#f
observation = [1, 6, 200, 100, 3100, 15.1, 99, 1, 200**2, 100**2, 3100**2, 15.1**2]
result2.predict(observation)[0]

38.73211109753365

The predicted miles per gallon is about 38.73 miles/gallon.

Problem 2

In [13]:
#a
# by python
import numpy as np
df = pd.DataFrame([[0, 3, 0, 'Red'], [2, 0, 0, 'Red'], [0, 1, 3, 'Red'], \
                   [0, 1, 2, 'Green'], [-1, 0, 1, 'Green'], [1, 1, 1, 'Red']])
df.columns = ['X1', 'X2', 'X3', 'Y']
df['Eucl. Dist. from X1 = X2 = X3 =0'] = np.sqrt(df['X1'] ** 2 + df['X2'] ** 2 + df['X3'] ** 2)
df.index+=1

df

,X1,X2,X3,Y,Eucl. Dist. from X1 = X2 = X3 =0
1,0,3,0,Red,3.000000
2,2,0,0,Red,2.000000
3,0,1,3,Red,3.162278
4,0,1,2,Green,2.236068
5,-1,0,1,Green,1.414214
6,1,1,1,Red,1.732051


In [15]:
#by hand

$$ dist(X_1, X_0) = \sqrt{(0 - 0)^2 + (3 - 0)^2 + (0 - 0)^2} = 3 $$
$$ dist(X_2, X_0) = \sqrt{(2 - 0)^2 + (0 - 0)^2 + (0 - 0)^2} = 2 $$
$$ dist(X_3, X_0) = \sqrt{(0 - 0)^2 + (1 - 0)^2 + (3 - 0)^2} = \sqrt{10} $$
$$ dist(X_4, X_0) = \sqrt{(0 - 0)^2 + (1 - 0)^2 + (2 - 0)^2} = \sqrt{5} $$
$$ dist(X_5, X_0) = \sqrt{(-1 - 0)^2 + (0 - 0)^2 + (1 - 0)^2} = \sqrt{2} $$
$$ dist(X_6, X_0) = \sqrt{(1 - 0)^2 + (1 - 0)^2 + (1 - 0)^2} = \sqrt{3} $$

In [ ]:
#b

As we can see from the above answer, when K = 1, the distance between X5 and X0 is least, so X0 should have the same color with X5, which is green.

In [ ]:
#c

When K = 3, we consider the points that have the least 3 distances, which are X5, X6, X2. And as we know, X6 and X2 are red while X5 is green, therefore, X0 should be red.

In [ ]:
#d

The expected value for K should be small. As our professor discussed in the lecture, if K is large, the boundary line will be much smoother, which is closer to linear compared with small K. If K is small, the boundary lines envelopes only a few points around each point, so it can be really non-linear.

In [18]:
#e
from sklearn import neighbors, linear_model
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

X_2 = [[0, 3, 0], [2, 0, 0], [0, 1, 3], [0, 1, 2], [-1, 0, 1], [1, 1, 1]]
Y = ['Red', 'Red', 'Red', 'Green', 'Green', 'Red']
knn = neighbors.KNeighborsClassifier(n_neighbors=2)
results = knn.fit(X_2, Y)
results.predict([[1, 1, 1]])[0]

'Green'

Problem 3

In [33]:
#a

In [42]:
df_quant1 = auto[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']]
df1 = sm.add_constant(df_quant1)
dummy = df1['mpg']>=df1['mpg'].median()
df1['mpg_high'] = np.where(dummy,1,0)

reg3 = sm.Logit(df1['mpg_high'], df1[['const', 'cylinders', \
                'displacement', 'horsepower', 'weight', 'acceleration' \
                ,'year', 'origin']], missing='drop')
results3 = reg3.fit()
print(results3.summary())

Optimization terminated successfully.
         Current function value: 0.200944
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:               mpg_high   No. Observations:                  392
Model:                          Logit   Df Residuals:                      384
Method:                           MLE   Df Model:                            7
Date:                Tue, 18 Feb 2020   Pseudo R-squ.:                  0.7101
Time:                        10:24:29   Log-Likelihood:                -78.770
converged:                       True   LL-Null:                       -271.71
Covariance Type:            nonrobust   LLR p-value:                 2.531e-79
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -17.1549      5.764     -2.976      0.003     -28.452      -5.858
cylinders       -0.1626

/Users/zhouanqi/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


The coefficients of constant term, weight variable and year variable are statistically significant at 5% level.

In [43]:
#b
from sklearn.model_selection import train_test_split
df1.dropna(inplace=True)
Y_3b = df1['mpg_high']
X_3b = df1[['cylinders', 'displacement','horsepower', 'weight', \
        'acceleration', 'year', 'origin']]

X_train, X_test, y_train, y_test = train_test_split(X_3b, Y_3b, \
                    test_size = 0.5, random_state=10)

In [61]:
#c
from sklearn.linear_model import LogisticRegression
results_3c = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, y_train)
print("beta0:", results_3c.intercept_[0])
k = 0
for i in results_3c.coef_[0]:
    k = k + 1
    print("beta" + str(k) +":", i)

beta0: -24.49498802358954
beta1: -1.1475143871562092
beta2: 0.016887155514053075
beta3: 0.014042771266676293
beta4: -0.007189770372789658
beta5: 0.14971820598867763
beta6: 0.5753090368866901
beta7: 0.13424597631409751


In [54]:
#d
y_hat = results_3c.predict(X_test)
print("The predicted values for test set is\n", y_hat)

The predicted values for test set is
 [1 1 0 1 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1
 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1
 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0
 0 0 1 0 0 1 0 0 1 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1
 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 1 1 0 1 1 0 1 1]


In [55]:
confusion_matrix(y_test, y_hat)

array([[85, 14],
       [ 9, 88]])

In [56]:
print(classification_report(y_hat, y_test,target_names=['low mpg', 'high mpg']))

              precision    recall  f1-score   support

     low mpg       0.86      0.90      0.88        94
    high mpg       0.91      0.86      0.88       102

    accuracy                           0.88       196
   macro avg       0.88      0.88      0.88       196
weighted avg       0.88      0.88      0.88       196



As we can see from the results, the f1 score is same for low_mpg and high_mpg. Meanwhile, precision and recall are all quite similar, so we can conclude that our model predicts low_mpg and high_mpg equally well.